In [1]:
import LinearAlgebra, DelimitedFiles
la = LinearAlgebra
df = DelimitedFiles;

In [2]:
function add_up_connections(A::Array{Float64, 2})
    pre_n = sum(convert(Array{Int}, A .!= 0); dims=2) # Number of connections from a given neuron
    pre_n[pre_n .== 0] .= 1                           # This is to remove a divide-by-zero; if n is 0, then so is A
    return pre_n
end

add_up_connections (generic function with 1 method)

In [3]:
function Θ(x::Array{Float64, 1}, x_rev::Float64, λ::Float64, θ::Float64)
    n_cortices = length(x)
    output = Array{Float64, 2}(undef, n_cortices, n_cortices)
    for ((k, xk), (j, xj)) in Iterators.product(enumerate(x), enumerate(x))
        output[j, k] = (xj - x_rev)/(1 + exp(-λ*(xk - θ)))
    end
    return output
end

Θ (generic function with 1 method)

In [4]:
function dΘ_dx(x::Array{Float64, 1}, λ::Float64, θ::Float64)
    n_cortices = length(x)
    final = ones(Float64, n_cortices, n_cortices)/(1 + exp(-λ*(x - θ)))
    diagonal = la.diagm(0 => x*λ*exp(-λ*(x - θ))/(1 + exp(-λ*(x - θ)))^2)
    return final + diagonal
end

dΘ_dx (generic function with 1 method)

In [5]:
function hr_dots(du, u, p, t)
    b, i0, x_rev, λ, θ, μ, s, x_rest, α, n1, β, n2, G1, G2 = p
    x, y, z = u
    theta = Θ(x, x_rev, λ, θ)
    du[1] = y .- (x^3) .+ b.*(x.^2) .+ i0 .- z .- (α./n1).*sum(G1.*theta, 2) - (β./n2).*sum(G2.*theta, 2)
    du[2] = 1 .- 5.*(x.^2) - y
    du[3] = μ.*(s.*(x .- s_rest) .- z)
    return du
end

hr_dots (generic function with 1 method)

In [6]:
const n = df.readdlm("../connectomes/cat_connectome");

In [7]:
N = size(n, 1)
pre_cortex_mask = zeros(Float64, N, N)
pre_cortex_mask[collect(1:17), collect(1:17)] .= 1
pre_cortex_mask[collect(17:27), collect(17:27)] .= 2
pre_cortex_mask[collect(27:45), collect(27:45)] .= 3
pre_cortex_mask[collect(45:65), collect(45:65)] .= 4
const cortex_mask = copy(pre_cortex_mask)
pre_G1 = copy(n)
pre_G1[cortex_mask .== 0] .= 0
pre_G2 = copy(n)
pre_G2[cortex_mask .!= 0] .= 0
const G1 = copy(pre_G1)
const G2 = copy(pre_G2);

In [8]:
const b = 3.2                       # Controls spiking frequency
const i0 = 4.4*ones(size(n, 2))     # Input current ---- It's an array so we can add noise later
const x_rev = 2                     # Reverse potential
const λ = 10                        # Sigmoidal function parameter
const θ = -0.25                     # Sigmoidal function parameter
const μ = 0.01                      # Time scale of slow current
const s = 4.0                       # Governs adaptation (whatever that means)
const x_rest = -1.6                 # Resting potential ------ INCORRECT IN SANTOS PAPER
const α = 0.210                     # Intra connection strength ---- VARIED PARAMETER
const n1 = add_up_connections(G1)   # Intra connections
const β = 0.040                     # Inter connection strength ---- VARIED PARAMETER
const n2 = add_up_connections(G2);  # Inter connections